# Merge Labels
TotalSegmentator v2 performs well on segmentation of most labels. However, MRSegmentator yields better results for Gallbladder, Autochtonal and Iliopsoas muscles (See [Notebook 4](04_EvaluateBaseline.ipynb)).
To achieve the best possible segmentations I use TotalSegmentator and replace these five labels by the segmentations of MRSegmentator.

I will procede as following:
1. Use Autochtonal and Iliopsoas Muscle segmentations of MRSegmentator
2. Add remaining labels of TotalSegmentator
3. Apply KeepLargestConnectedComponent to Autochtonal and Iliopsoas muscles*

*If segementations overlap, those of TotalSegmentator have priority over MRSegmentator. In some cases this might result in a splitted segementation (e.g. one left iliopsoas muscle segmentation gets split into two segmentations). In those cases I will only keep the larger segmentation. 

This script is only for ilustartion, the full source code is in [map_merge.py](../scripts/map_merge.py).

In [2]:
from monai.transforms import (
    SaveImage,
    KeepLargestConnectedComponent,
    MapLabelValue,
)
from tqdm.notebook import tqdm

# private libraries
import sys

if "../scripts" not in sys.path:
    sys.path.insert(1, "../scripts")
import labelmappings as lm

In [ ]:
deleteMusclesGall = MapLabelValue(
    orig_labels=[4, 31, 32, 33, 34],
    target_labels=[0, 0, 0, 0, 0],
    dtype=int,
)
deleteAllButMusclesGall = MapLabelValue(
    orig_labels=list(range(4)) + list(range(5, 31)) + list(range(35, 41)),
    target_labels=[0] * 36,
    dtype=int,
)

keepLargestMusclesGall = KeepLargestConnectedComponent(applied_labels=[4, 31, 32, 33, 34])

save = SaveImage(
    output_dir="update_path",
    separate_folder=False,
    output_postfix="new",
)

In [ ]:
def merge_label(totalseg_label, mrseg_label):
    # clean labels
    totalseg_label = deleteMusclesGall(totalseg_label)
    mrseg_label = deleteAllButMusclesGall(mrseg_label)

    # merge labels
    new_label = mrseg_label.detach().clone()
    new_label[totalseg_label != 0] = totalseg_label[totalseg_label != 0]

    # remove artifacts that were created during merging
    new_label = keepLargestMusclesGall(new_label)
    return new_label

In [3]:
mr_labels = []  # List of segmentations by MrSegmentator
total_labels = []  # List of segmentations by TotalSegmentator
for i in tqdm(range(len(mr_labels))):
    # Load labels and map to new metadata specification
    label_mr = lm.transform_mr(mr_labels[i])
    label_total = lm.transform_total(total_labels[i])

    # merge labels
    new_label = merge_label(label_total, label_mr)

    # save label
    save(new_label)

0it [00:00, ?it/s]